# The Other Side of the Lenss.

## Abstract ciaoooooo


Photojournalists are a backbone of photography, they carry out a job that can hardly be replaced because they testify to the facts, events and changes that take place around us. The issue is that they are always less famous as they diserve to be. In this study we interrogate ourself if this behaviour is throuth or not just by looking to data. 

We took data from World Press Photo Archive, where we took information about awarded photographs and their authors, and artsy.net, a art market place, from which we took info about photographs and photorgaphers. 

Missing information about photographer pubblications, awards and links are taken from wikidata api. 


***Get Data***

In order to take data about photographs we scrape the World Press Photo. We know that photographer has the world person in the URI, so we select any photographer's link from the World Press Photo sitemap.

In [2]:
import requests
from bs4 import BeautifulSoup
import pprint as pp

sitemap = '{"_id":"wordpressphoto","startUrl":["https://www.worldpressphoto.org/collection/photo-contest"],"selectors":[{"id":"scroll","parentSelectors":["_root"],"type":"SelectorElementScroll","selector":"div.swiper-container","multiple":true,"delay":5000},{"id":"link","parentSelectors":["_root"],"type":"SelectorLink","selector":"div:nth-of-type(n+3) a","multiple":true,"delay":0},{"id":"next","parentSelectors":["_root"],"type":"SelectorElementClick","clickElementSelector":"div.swiper-btn--next","clickElementUniquenessType":"uniqueText","clickType":"clickOnce","delay":2000,"discardInitialElements":"do-not-discard","multiple":true,"selector":"div.swiper-btn--next"},{"id":"title","parentSelectors":["link"],"type":"SelectorText","selector":"h1","multiple":false,"delay":0,"regex":""},{"id":"photographer","parentSelectors":["link"],"type":"SelectorText","selector":"h2.title--white","multiple":false,"delay":0,"regex":""},{"id":"shoot day","parentSelectors":["link"],"type":"SelectorText","selector":".photo-details__section--general-info p:nth-of-type(1)","multiple":false,"delay":0,"regex":""},{"id":"genere","parentSelectors":["link"],"type":"SelectorText","selector":"p.photo-details","multiple":false,"delay":0,"regex":""},{"id":"award","parentSelectors":["link"],"type":"SelectorText","selector":".photo-details__fixed-column-title h5","multiple":false,"delay":0,"regex":""},{"id":"description","parentSelectors":["link"],"type":"SelectorText","selector":"div.photo-details__section--general-info","multiple":false,"delay":0,"regex":""},{"id":"Shutter Speed","parentSelectors":["link"],"type":"SelectorText","selector":"div.photo-details__feature--shutter","multiple":false,"delay":0,"regex":""},{"id":"Focal length","parentSelectors":["link"],"type":"SelectorText","selector":"div.photo-details__feature--length","multiple":false,"delay":0,"regex":""},{"id":"F-Stop","parentSelectors":["link"],"type":"SelectorText","selector":"div.photo-details__feature--aperture","multiple":false,"delay":0,"regex":""},{"id":"ISO","parentSelectors":["link"],"type":"SelectorText","selector":"div.photo-details__feature--iso","multiple":false,"delay":0,"regex":""},{"id":"Camera","parentSelectors":["link"],"type":"SelectorText","selector":"div.photo-details__feature--camera","multiple":false,"delay":0,"regex":""}]}'

url = 'https://www.worldpressphoto.org/sitemap.xml'

r = requests.get(url)

sp = BeautifulSoup(r.text, 'html')

links = sp.find_all('loc')
print(len(links))

photographers = []

for link in links:
    if 'person' in link.text:
        photographers.append(str(link.text))
print(photographers)




2991
['https://www.worldpressphoto.org/person/detail/593/a-new-age-of-walls-team', 'https://www.worldpressphoto.org/person/detail/594/a-wildenberg', 'https://www.worldpressphoto.org/person/detail/595/aapo-huhta', 'https://www.worldpressphoto.org/person/detail/596/aaron-goodman', 'https://www.worldpressphoto.org/person/detail/597/aaron-huey', 'https://www.worldpressphoto.org/person/detail/41993/abbas-kowsari', 'https://www.worldpressphoto.org/person/detail/598/abbie-trayler-smith', 'https://www.worldpressphoto.org/person/detail/39116/abby-nicolas', 'https://www.worldpressphoto.org/person/detail/599/abd-doumany', 'https://www.worldpressphoto.org/person/detail/600/abdellah-azizi', 'https://www.worldpressphoto.org/person/detail/601/abdolla-heidari', 'https://www.worldpressphoto.org/person/detail/602/abdullah-doma', 'https://www.worldpressphoto.org/person/detail/603/abdurrauf-ben-madi', 'https://www.worldpressphoto.org/person/detail/604/abir-abdullah', 'https://www.worldpressphoto.org/perso

Than we storage all the information provided by all the links in CSV format. First of all we extract the name of photographers.

In [17]:
import csv
with open('worldpressphotographer.csv', mode='w') as my_file:
    my_writer = csv.writer(my_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
    # write the column names
    my_writer.writerow(['photographer', 'nationality', 'description'])

    for url in photographers:
        page = requests.get(url)
        
        soup = BeautifulSoup(page.content, "html.parser")

        for photographer in soup.select('div.text-widget h1'):
            pho_name = photographer.text

        n = 'none'
        for nationality in soup.select('div.paragraph'):
            if nationality:
                n = nationality.text
           

        txt = []
        for description in soup.select('div.text-widget p'):
            txt.append(description.text)

        full_txt = " ".join(txt)
       

        my_writer.writerow([pho_name, n, full_txt])



In [50]:
import pandas as pd
# parse the csv into a dataframe
data = pd.read_csv("worldpressphotographer.csv")
# print the first 5 rows
data.head()

,photographer,nationality,description
0,A New Age of Walls team,none,"In the United States, Donald Trump’s call to b..."
1,A. Wildenberg,France,NaN
2,Aapo Huhta,Finland,Aapo Huhta received his MA in photography from...
3,Aaron Goodman,none,His research and documentary production focus ...
4,Aaron Huey,USA,Aaron Huey is a National Geographic photograph...


In [58]:
import spacy
from spacy import displacy

NER = spacy.load("en_core_web_sm")

with open('worldpressphotographer.csv', encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')

    df = pd.read_csv("worldpressphotographer.csv")

    i = 0

    for row in csv_reader:
        i += 1

        if row[2] != "description":

            full_txt = row[2]
            parsed = NER(full_txt)
            person = set()
            organization = set()
            place = set()
            for word in parsed.ents:
                
                if word.label_ == 'PERSON':
                    person.add(word.text)
                if word.label_ == 'ORG':
                    organization.add(word.text)
                if word.label_ == 'GPE':
                    place.add(word.text)

                df.at[i, "persons"] = str(person)
                df.at[i, "tot_persons"] = len(person)
                df.at[i, "organizations"] = str(organization)
                df.at[i, "tot_organizations"] = len(organization)
                df.at[i, "places"] = str(place)
                df.at[i, "tot_places"] = len(place)
                df.to_csv("worldpressphotographer_update.csv", index=False)
              

In [59]:
data = pd.read_csv("worldpressphotographer_update.csv")
# print the first 5 rows
data.head(50)

,photographer,nationality,description,persons,tot_persons,organizations,tot_organizations,places,tot_places
0,A New Age of Walls team,none,"In the United States, Donald Trump’s call to b...",NaN,NaN,NaN,NaN,NaN,NaN
1,A. Wildenberg,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Aapo Huhta,Finland,Aapo Huhta received his MA in photography from...,"{'Stephanie Kirchner', 'Ruth Eglash', 'Douglas...",12.0,"{'The Washington Post’s', 'The Washington Post'}",2.0,"{'US', 'Mexico', 'the United States', 'Israel'...",5.0
3,Aaron Goodman,none,His research and documentary production focus ...,NaN,NaN,NaN,NaN,NaN,NaN
4,Aaron Huey,USA,Aaron Huey is a National Geographic photograph...,{'Aapo Huhta'},1.0,"{'MA', 'the\xa0Young Nordic Photographer of th...",4.0,"{'Stockholm', 'Finland', 'Berlin'}",3.0
5,Abbas Kowsari,Iran,Abbas Kowsari is the senior photo editor for S...,"{'Calgary Herald', 'Alert Net'}",2.0,"{'CNN', 'PBS Frontline/WORLD', 'VOA', 'Associa...",12.0,set(),0.0
6,Abbie Trayler-Smith,UK,Abbie Trayler-Smith is a documentary and portr...,{'Aaron Huey'},1.0,"{'Harper', 'National Geographic', 'Harper’s Ma...",3.0,"{'USA', 'America', 'Seattle'}",3.0
7,Abby Nicolas,USA,Abby Nicolas is a photo editor on staff at Spo...,"{'Ag Galerie', 'Kowsari', 'Der Spiegel', 'Abba...",7.0,"{'Aftab Network Magazine', 'Social Media', 'th...",12.0,"{'Tehran', 'Iran', 'London'}",3.0
8,Abd Doumany,none,"Before the war broke out, Doumany was completi...","{'Abbie Trayler-Smith', 'Welsh', 'Taylor Wessi...",3.0,"{'Panos Pictures', 'The National Portrait Gall...",8.0,"{'London', 'UK', 'South Wales', 'Iraq', 'Darfur'}",5.0
9,Abdellah Azizi,Morocco,"Abdellah Azizi (Morocco, 1987) studied cinemat...","{'Taylor Ballantyne', 'Abby Nicolas', '@crabby...",4.0,"{'Time Inc.', 'Parsons School of Design', 'Pen...",8.0,"{'US', 'New York', 'Brooklyn', 'New York City'...",5.0


In [ ]:
import pandas_profiling as pp
pp.ProfileReport(data)

Than we took info from collection World Press Photo by scaping with Web Scaper. Below sitemap for collection. And we result with a csv file wordpressphoto_data.csv.

In [61]:

sitemap = '{"_id":"wordpressphoto","startUrl":["https://www.worldpressphoto.org/collection/photo-contest"],"selectors":[{"delay":5000,"id":"scroll","multiple":true,"parentSelectors":["_root"],"selector":"div.swiper-container","type":"SelectorElementScroll"},{"delay":0,"id":"link","multiple":true,"parentSelectors":["_root"],"selector":"div:nth-of-type(n+3) a","type":"SelectorLink"},{"clickElementSelector":"div.swiper-btn--next","clickElementUniquenessType":"uniqueText","clickType":"clickOnce","delay":2000,"discardInitialElements":"do-not-discard","id":"next","multiple":true,"parentSelectors":["_root"],"selector":"div.swiper-btn--next","type":"SelectorElementClick"},{"delay":0,"id":"title","multiple":false,"parentSelectors":["link"],"regex":"","selector":"h1","type":"SelectorText"},{"delay":0,"id":"photographer","multiple":false,"parentSelectors":["link"],"regex":"","selector":"h2.title--white","type":"SelectorText"},{"delay":0,"id":"shoot day","multiple":false,"parentSelectors":["link"],"regex":"","selector":".photo-details__section--general-info p:nth-of-type(1)","type":"SelectorText"},{"delay":0,"id":"Publication agency/magazine","multiple":false,"parentSelectors":["link"],"regex":"","selector":"p.photo-details","type":"SelectorText"},{"delay":0,"id":"award","multiple":false,"parentSelectors":["link"],"regex":"","selector":".photo-details__fixed-column-title h5","type":"SelectorText"},{"delay":0,"id":"description","multiple":false,"parentSelectors":["link"],"regex":"","selector":"div.photo-details__section--general-info","type":"SelectorText"},{"delay":0,"id":"Shutter Speed","multiple":false,"parentSelectors":["link"],"regex":"","selector":"div.photo-details__feature--shutter","type":"SelectorText"},{"delay":0,"id":"Focal length","multiple":false,"parentSelectors":["link"],"regex":"","selector":"div.photo-details__feature--length","type":"SelectorText"},{"delay":0,"id":"F-Stop","multiple":false,"parentSelectors":["link"],"regex":"","selector":"div.photo-details__feature--aperture","type":"SelectorText"},{"delay":0,"id":"ISO","multiple":false,"parentSelectors":["link"],"regex":"","selector":"div.photo-details__feature--iso","type":"SelectorText"},{"delay":0,"id":"Camera","multiple":false,"parentSelectors":["link"],"regex":"","selector":"div.photo-details__feature--camera","type":"SelectorText"}]}'

df = pd.read_csv("wordpressphoto_data.csv")

df.head(15)

ParserError: Error tokenizing data. C error: Expected 4 fields in line 7, saw 28


Artsy.net API

Wikidata riconciliation. 

In [1]:
#!pip install qwikidata
import pprint
import requests
from qwikidata.sparql  import return_sparql_query_results

pp = pprint.PrettyPrinter(indent=1)

def wikidata_reconciliation(query, q_class=None):
    print(query)
    """ query wd apis and print in a json file the results of reconciliation """
    for q in query:
        print(q)
        API_WD = "https://www.wikidata.org/w/api.php"
        params = {
            'action': 'wbsearchentities',
            'format': 'json',
            'language': 'en',
            'search': q # the query string
        }
        
        # query wd API    
        r = requests.get(API_WD, params = params).json() 
        pp.pprint(r) # the response
        
        # iterate over results (if there is any)
        if 'search' in r and len(r['search']) >= 1:
            # if specified, double check if the entity belongs to the class
            if q_class:
                qid= r['search'][0]['title']
                query_string = """ASK {wd:"""+qid+""" wdt:P31 wd:"""+q_class+""". }"""
                
                # query WD endpoint this time!
                res = return_sparql_query_results(query_string) 
                print("\n my string:", query, "\n the query to WD endpoint:", query_string, "\n the result:",res)
                
                if res["boolean"] == True: 
                    print( r['search'][0]['title'] , 'the class matches :)')
                else:
                    print( r['search'][0]['title'] , 'the class does not match :(')
            else:
                print( r['search'][0]['title'] , 'no class was given')
        else:
            print( 'no results matching the query string')

In [ ]:
fotografi = ["Mads Nissen", "Antonio Faccilongo"]

wikidata_reconciliation(fotografi, "Q5") 
#passare lista fotografi unici